In [154]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [155]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

In [156]:
import os, tempfile, glob, random
from pathlib import Path
from IPython.display import Markdown
from PIL import Image
from getpass import getpass
import numpy as np
from itertools import combinations

# LLM: openai and google_genai
import openai
from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# LLM: HuggingFace
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.llms import HuggingFaceHub

# langchain prompts, memory, chains...
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain.schema import Document, format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
)

# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# OutputParser
from langchain_core.output_parsers import StrOutputParser

# Chroma: vectorstore
from langchain_community.vectorstores import Chroma

# Contextual Compression
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

# Cohere
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere

In [157]:
# Data Directories: where temp files and vectorstores will be saved

TMP_DIR = Path("./data").resolve().parent.joinpath("data", "tmp")
LOCAL_VECTOR_STORE_DIR = Path("./data").resolve().parent.joinpath("data", "vector_stores")

In [158]:
import os
from getpass import getpass
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

def get_environment_variable(key):
    if key in os.environ:
        value = os.environ.get(key)
        print(f"\n[INFO]: {key} retrieved successfully.")
    else:
        print(f"\n[ERROR]: {key} is not found in your environment variables.")
        value = getpass(f"Insert your {key}: ")
    return value

HF_key = get_environment_variable("HUGGINGFACEHUB_API_TOKEN")
cohere_api_key = get_environment_variable("COHERE_API_KEY")
google_api_key = get_environment_variable("GOOGLE_API_KEY")



[INFO]: HUGGINGFACEHUB_API_TOKEN retrieved successfully.

[INFO]: COHERE_API_KEY retrieved successfully.

[INFO]: GOOGLE_API_KEY retrieved successfully.


In [160]:
def langchain_document_loader(TMP_DIR):
    """
    Load files from TMP_DIR (temporary directory) as documents. Files can be in txt, pdf, CSV or docx format.
    https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory
    """

    documents = []

    txt_loader = DirectoryLoader(
        TMP_DIR.as_posix(), glob="**/*.txt", loader_cls=TextLoader, show_progress=True
    )
    documents.extend(txt_loader.load())

    pdf_loader = DirectoryLoader(
        TMP_DIR.as_posix(), glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True
    )
    documents.extend(pdf_loader.load())

    csv_loader = DirectoryLoader(
        TMP_DIR.as_posix(), glob="**/*.csv", loader_cls=CSVLoader, show_progress=True,
        loader_kwargs={"encoding":"utf8"}
    )
    documents.extend(csv_loader.load())

    doc_loader = DirectoryLoader(
        TMP_DIR.as_posix(),
        glob="**/*.docx",
        loader_cls=Docx2txtLoader,
        show_progress=True,
    )
    documents.extend(doc_loader.load())
    return documents

In [161]:
documents = langchain_document_loader(TMP_DIR)
print(f"\nNumber of documents: {len(documents)}")

0it [00:00, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  1.83it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


Number of documents: 18


In [162]:
document_names = [doc.metadata.get('source') for doc in documents] 

print("\nDocument Names:")
for name in document_names:
    print(name)


Document Names:
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag_langchain_chatbot\data\tmp\2024-01-24-VirtualTryAll.pdf
C:\d drive\Anurag\MAchine Learning\Projects\Rag

In [164]:
import random
random_document_id = random.choice(range(len(documents)))

Markdown(f"**Document[{random_document_id}]** \n\n **Page content** (first 1000 character):\n\n" +\
         documents[random_document_id].page_content[0:1000] + " ..."  +\
         "\n\n**Metadata:**\n\n" + str(documents[random_document_id].metadata))

**Document[7]** 

 **Page content** (first 1000 character):

Table 1. Quantitative comparison between DTC variants and
PBE best, which denotes a PBE variant using DINOv2 and percep-
tual loss. CA denotes Cross-Attention.
Method CLIP Score ( ↑) FID ( ↓)
PBE best 85.43 6.65
Ours addition 86.94 6.19
Ours CA 88.01 5.68
Ours FiLM 88.14 5.72
integration of more computationally expensive Cross Atten-
tion layers [43]. Results shown on Tab. 1 revealed that both
FiLM and Cross Attention layers outperform direct addi-
tion. Also, Cross Attention and FiLM yield comparable re-
sults, and FiLM is cheaper to compute, therefore we chose
to use FiLM in our final model.
Figure 7. Failure cases with generating fine-grained text.
4.4. Evaluation and Comparisons
Comparison Against Paint by Example Variants. We
implemented a series of enhancements to PBE and trained
each variant on VITONHD-NoFace dataset. The results are
presented in Table 2. As anticipated, using all CLIP patches
surpasses the performance of using only the [CLS] token,
which is limited to encoding  ...

**Metadata:**

{'source': 'C:\\d drive\\Anurag\\MAchine Learning\\Projects\\Rag_langchain_chatbot\\data\\tmp\\2024-01-24-VirtualTryAll.pdf', 'page': 7}

# Text Splitters

In [165]:

# Create a RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", ""],    
    chunk_size = 1600,
    chunk_overlap= 200
)

# Text splitting
chunks = text_splitter.split_documents(documents=documents)
print(f"number of chunks: {len(chunks)}")

number of chunks: 45


In [166]:
import tiktoken
def tiktoken_tokens(documents,model="gpt-3.5-turbo"):
    """Use tiktoken (tokeniser for OpenAI models) to return a list of token lengths per document."""    
    encoding = tiktoken.encoding_for_model(model) # returns the encoding used by the model.
    
    tokens_length = [len(encoding.encode(documents[i].page_content)) for i in range(len(documents))]

    return tokens_length

In [167]:

chunks_length = tiktoken_tokens(chunks,model="gpt-3.5-turbo")

print(f"Number of tokens - Average : {int(np.mean(chunks_length))}")
print(f"Number of tokens - 25% percentile : {int(np.quantile(chunks_length,0.25))}")
print(f"Number of tokens - 50% percentile : {int(np.quantile(chunks_length,0.5))}")
print(f"Number of tokens - 75% percentile : {int(np.quantile(chunks_length,0.75))}")
print("\nMax_tokens for gpt-3.5-turbo: 4096")

Number of tokens - Average : 291
Number of tokens - 25% percentile : 165
Number of tokens - 50% percentile : 350
Number of tokens - 75% percentile : 373

Max_tokens for gpt-3.5-turbo: 4096


In [227]:


def select_embeddings_model(LLM_service="OpenAI"):
    """Connect to the embeddings API endpoint by specifying the name of the embedding model."""
    if LLM_service == "OpenAI":
        embeddings = OpenAIEmbeddings(
            model='text-embedding-ada-002',
            api_key=openai_api_key)

    if LLM_service == "Google":
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=google_api_key
        )
    
         
    return embeddings

embeddings_google = select_embeddings_model(LLM_service="Google")


In [228]:


sentences = ["I like pets.",
             "Pets bring joy to our lives.",
             "Langchain is a framework for developing applications powered by LLMs."]
# 1. Calculate embedding vectors
embedding_vectors = [embeddings_google.embed_query(sentence) for sentence in sentences]

for combination in list(combinations(range(len(sentences)),2)):
    # 2. Calculate similarity using dot product from numpy:
    dot_prodduct = round(np.dot(embedding_vectors[combination[0]], embedding_vectors[combination[1]]),3)
    print(f"Similarty of sentences {combination}: {dot_prodduct}")

Similarty of sentences (0, 1): 0.904
Similarty of sentences (0, 2): 0.679
Similarty of sentences (1, 2): 0.687


In [229]:
vector_store_google = Chroma(
    persist_directory = LOCAL_VECTOR_STORE_DIR.as_posix() + "/Vit_All_Google_Embeddings",
    embedding_function=embeddings_google)
print("vector_store_google:",vector_store_google._collection.count(),"chunks.")


vector_store_google: 540 chunks.


In [230]:
def create_vectorstore(embeddings, documents, vectorstore_name):
    """Create a Chroma vector database."""
    persist_directory = (LOCAL_VECTOR_STORE_DIR.as_posix() + "/" + vectorstore_name)
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vector_store, vectorstore_name  # Return the original vectorstore_name

create_vectorstores = True 

if create_vectorstores:
    vector_store_google, _ = create_vectorstore(
        embeddings=embeddings_google,
        documents=chunks,
        vectorstore_name="Vit_All_Google_Embeddings"
    )
    print("vector_store_google:", vector_store_google._collection.count(), "chunks.")
    
    

vector_store_google: 585 chunks.


In [231]:
vector_store_google = Chroma(
    persist_directory = LOCAL_VECTOR_STORE_DIR.as_posix() + "/Vit_All_Google_Embeddings",
    embedding_function=embeddings_google)
print("vector_store_google:",vector_store_google._collection.count(),"chunks.")



vector_store_google: 585 chunks.


In [232]:
def print_documents(docs,search_with_score=False):
    """helper function to print documents."""
    if search_with_score:
        # used for similarity_search_with_score
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc[0].page_content +"\n\nscore:"+str(round(doc[-1],3))+"\n" 
                 for i, doc in enumerate(docs)]
            )
        )
    else:
        # used for similarity_search or max_marginal_relevance_search
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc.page_content 
                 for i, doc in enumerate(docs)]
            )
        )

In [233]:
query = 'What is Diffuse to choose?'
docs_withScores = vector_store_google.similarity_search_with_score(query,k=4)

print_documents(docs_withScores,search_with_score=True)

Document 1:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024

score:0.521

----------------------------------------------------------------------------------------------------
Document 2:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024

score:0.521

-----------------------------------------------

In [234]:
# Here the scores are dot products. So a higher score is better.

query_embeddings = embeddings_google.embed_query(query)
docs_embeddings = embeddings_google.embed_documents(
    [docs_withScores[i][0].page_content 
     for i in range(len(docs_withScores))
    ]
)

for i in range(len(docs_embeddings)):
    dot_product = round(np.dot(query_embeddings, docs_embeddings[i]),4)
    print(f"Similarty of document_{i} to the query: {dot_product}")

Similarty of document_0 to the query: 0.7394
Similarty of document_1 to the query: 0.7394
Similarty of document_2 to the query: 0.7394
Similarty of document_3 to the query: 0.7394


In [235]:

def Vectorstore_backed_retriever(vectorstore,search_type="similarity",k=4,score_threshold=None):
    """create a vectorsore-backed retriever
    Parameters: 
        search_type: Defines the type of search that the Retriever should perform.
            Can be "similarity" (default), "mmr", or "similarity_score_threshold"
        k: number of documents to return (Default: 4) 
        score_threshold: Minimum relevance threshold for similarity_score_threshold (default=None)
    """
    search_kwargs={}
    if k is not None:
        search_kwargs['k'] = k
    if score_threshold is not None:
        search_kwargs['score_threshold'] = score_threshold

    retriever = vectorstore.as_retriever(
        search_type=search_type,
        search_kwargs=search_kwargs
    )
    return retriever

In [236]:
# similarity search
base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"similarity",k=10)


In [237]:

# Get relevant documents

query = 'what is Diffuse to Choose?'
relevant_docs = base_retriever_google.get_relevant_documents(query)

print_documents(relevant_docs)

Document 1:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024
----------------------------------------------------------------------------------------------------
Document 2:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024
---------------------------------------------------------------------------

In [238]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

def create_compression_retriever(embeddings, base_retriever, chunk_size=500, k=16, similarity_threshold=None):
    """Build a ContextualCompressionRetriever.
    We wrap the the base_retriever (a vectorstore-backed retriever) into a ContextualCompressionRetriever.
    The compressor here is a Document Compressor Pipeline, which splits documents
    into smaller chunks, removes redundant documents, filters out the most relevant documents,
    and reorder the documents so that the most relevant are at the top and bottom of the list.
    
    Parameters:
        embeddings: OpenAIEmbeddings, GoogleGenerativeAIEmbeddings or HuggingFaceInferenceAPIEmbeddings.
        base_retriever: a vectorstore-backed retriever.
        chunk_size (int): Documents will be splitted into smaller chunks using a CharacterTextSplitter with a default chunk_size of 500. 
        k (int): top k relevant chunks to the query are filtered using the EmbeddingsFilter. default =16.
        similarity_threshold : minimum relevance threshold used by the EmbeddingsFilter.. default =None.
    """
    
    # 1. splitting documents into smaller chunks
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, separator=". ")
    
    # 2. removing redundant documents
    redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

    # 3. filtering based on relevance to the query    
    relevant_filter = EmbeddingsFilter(embeddings=embeddings, k=k, similarity_threshold=similarity_threshold) # similarity_threshold and top K

    # 4. Reorder the documents 
    
    # Less relevant document will be at the middle of the list and more relevant elements at the beginning or end of the list.
    # Reference: https://python.langchain.com/docs/modules/data_connection/retrievers/long_context_reorder
    reordering = LongContextReorder()

    # 5. Create compressor pipeline and retriever
    
    pipeline_compressor = DocumentCompressorPipeline(
        transformers=[splitter, redundant_filter, relevant_filter, reordering]  
    )
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=pipeline_compressor, 
        base_retriever=base_retriever
    )

    return compression_retriever

In [239]:
compression_retriever_Google = create_compression_retriever(
    embeddings=embeddings_google,
    base_retriever=base_retriever_google,
    k=16)

query = 'what is Diffuse to Choose?'

compressed_docs = compression_retriever_Google.get_relevant_documents(query)

print_documents(compressed_docs)

Document 1:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024


In [240]:

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere


In [241]:
def CohereRerank_retriever(
    base_retriever, 
    cohere_api_key,cohere_model="rerank-multilingual-v2.0", top_n=8
):
    """Build a ContextualCompressionRetriever using Cohere Rerank endpoint to reorder the results based on relevance.
    Parameters:
       base_retriever: a Vectorstore-backed retriever
       cohere_api_key: the Cohere API key
       cohere_model: The Cohere model can be either 'rerank-english-v2.0' or 'rerank-multilingual-v2.0', with the latter being the default.
       top_n: top n results returned by Cohere rerank, default = 8.
    """
    
    compressor = CohereRerank(
        cohere_api_key=cohere_api_key, 
        model=cohere_model, 
        top_n=top_n
    )

    retriever_Cohere = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )
    return retriever_Cohere

In [242]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API token
cohere_api_key = os.getenv("COHERE_API_KEY")



retriever_Cohere_google = CohereRerank_retriever(
    base_retriever=base_retriever_google, 
    cohere_api_key=cohere_api_key, cohere_model="rerank-multilingual-v2.0",  
    top_n=8
)

query = 'what is Diffuse to Choose?'

docs_cohere = retriever_Cohere_google.get_relevant_documents(query)

print("Reranking using Cohere. Vcetorstore using Google Embeddings:\n")
print_documents(docs_cohere)

Reranking using Cohere. Vcetorstore using Google Embeddings:

Document 1:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024
----------------------------------------------------------------------------------------------------
Document 2:

on incorporating fine-grained features from the reference
image directly into the latent feature maps of the main dif-
fusion model, alongside with a perceptual loss to further
preserve the reference item’s details. We conduct extensive
testing on both in-house and publicly available datasets,
and show that Diffuse to Choose is superior to existing zero-
1arXiv:2401.13795v1  [cs.CV]  24 Jan 2024
-------------

In [243]:
def retrieval_blocks(
    create_vectorstore=True,# if True a Chroma vectorstore is created, else the Chroma vectorstore will be loaded
    LLM_service="Google",
    vectorstore_name="Vit_All_google_Embeddings",
    chunk_size = 1600, chunk_overlap=200, # parameters of the RecursiveCharacterTextSplitter
    retriever_type="Vectorstore_backed_retriever",
    base_retriever_search_type="similarity", base_retriever_k=10, base_retriever_score_threshold=None,
    compression_retriever_k=16,
    cohere_api_key=cohere_api_key, cohere_model="rerank-multilingual-v2.0", cohere_top_n=8,
):
    """
    Rertieval includes: document loaders, text splitter, vectorstore and retriever. 
    
    Parameters: 
        create_vectorstore (boolean): If True, a new Chroma vectorstore will be created. Otherwise, an existing vectorstore will be loaded.
        LLM_service: OpenAI, Google or HuggingFace.
        vectorstore_name (str): the name of the vectorstore.
        chunk_size and chunk_overlap: parameters of the RecursiveCharacterTextSplitter, default = (1600,200).
        
        retriever_type (str): in [Vectorstore_backed_retriever,Contextual_compression,Cohere_reranker]
        
        base_retriever_search_type: search_type in ["similarity", "mmr", "similarity_score_threshold"], default = similarity.
        base_retriever_k: The most similar vectors to retrieve (default k = 10).  
        base_retriever_score_threshold: score_threshold used by the base retriever, default = None.

        compression_retriever_k: top k documents returned by the compression retriever, default=16
        
        cohere_api_key: Cohere API key
        cohere_model (str): The Cohere model can be either 'rerank-english-v2.0' or 'rerank-multilingual-v2.0', with the latter being the default.
        cohere_top_n: top n results returned by Cohere rerank, default = 8.
   
    Output:
        retriever.
    """
    try:
        # Create new Vectorstore (Chroma index)
        if create_vectorstore: 
            # 1. load documents
            documents = langchain_document_loader(TMP_DIR)
            
            # 2. Text Splitter: split documents to chunks
            text_splitter = RecursiveCharacterTextSplitter(
                separators = ["\n\n", "\n", " ", ""],    
                chunk_size = chunk_size,
                chunk_overlap= chunk_overlap
            )
            chunks = text_splitter.split_documents(documents=documents)
            
            # 3. Embeddings
            embeddings = select_embeddings_model(LLM_service=LLM_service)
        
            # 4. Vectorsore: create Chroma index
            vector_store = create_vectorstore(
                embeddings=embeddings,
                documents = chunks,
                vectorstore_name=vectorstore_name,
            )
    
        # 5. Load a Vectorstore (Chroma index)
        else: 
            embeddings = select_embeddings_model(LLM_service=LLM_service)        
            vector_store = Chroma(
                persist_directory = LOCAL_VECTOR_STORE_DIR.as_posix() + "/" + vectorstore_name,
                embedding_function=embeddings
            )
            
        # 6. base retriever: Vector store-backed retriever 
        base_retriever = Vectorstore_backed_retriever(
            vector_store,
            search_type=base_retriever_search_type,
            k=base_retriever_k,
            score_threshold=base_retriever_score_threshold
        )
        retriever = None
        if retriever_type=="Vectorstore_backed_retriever": 
            retriever = base_retriever
    
        # 7. Contextual Compression Retriever
        if retriever_type=="Contextual_compression":    
            retriever = create_compression_retriever(
                embeddings=embeddings,
                base_retriever=base_retriever,
                k=compression_retriever_k,
            )
    
        # 8. CohereRerank retriever
        if retriever_type=="Cohere_reranker":
            retriever = CohereRerank_retriever(
                base_retriever=base_retriever, 
                cohere_api_key=cohere_api_key, 
                cohere_model=cohere_model, 
                top_n=cohere_top_n
            )
    
        print(f"\n{retriever_type} is created successfully!")
        print(f"Relevant documents will be retrieved from vectorstore ({vectorstore_name}) which uses {LLM_service} embeddings \
and has {vector_store._collection.count()} chunks.")
        
        return retriever
    except Exception as e:
        print(e)

## LLM


In [244]:
def instantiate_LLM(LLM_provider,api_key,temperature=0.5,top_p=0.95,model_name=None):
    """Instantiate LLM in Langchain.
    Parameters:
        LLM_provider (str): the LLM provider; in ["OpenAI","Google","HuggingFace"]
        model_name (str): in ["gpt-3.5-turbo", "gpt-3.5-turbo-0125", "gpt-4-turbo-preview", 
            "gemini-pro", "mistralai/Mistral-7B-Instruct-v0.2"].            
        api_key (str): google_api_key or openai_api_key or huggingfacehub_api_token 
        temperature (float): Range: 0.0 - 1.0; default = 0.5
        top_p (float): : Range: 0.0 - 1.0; default = 1.
    """
    if LLM_provider == "OpenAI":
        llm = ChatOpenAI(
            api_key=api_key,
            model=model_name,
            temperature=temperature,
            model_kwargs={
                "top_p": top_p
            }
        )
    if LLM_provider == "Google":
        llm = ChatGoogleGenerativeAI(
            google_api_key=api_key,
            # model="gemini-pro",
            model=model_name,
            temperature=temperature,
            top_p=top_p,
            convert_system_message_to_human=True
        )
    if LLM_provider == "HuggingFace":
        llm = HuggingFaceHub(
            repo_id="mistralai/Mistral-7B-Instruct-v0.3", 
            #repo_id=model_name,
            huggingfacehub_api_token=api_key,
            model_kwargs={
                "temperature":temperature,
                "top_p": top_p,
                "do_sample": True,
                "max_new_tokens":1024
            },
        )
    return llm

In [245]:
from langchain.memory import ConversationSummaryBufferMemory,ConversationBufferMemory

def create_memory(model_name='gpt-3.5-turbo',memory_max_token=None):
    """Creates a ConversationSummaryBufferMemory for gpt-3.5-turbo
    Creates a ConversationBufferMemory for the other models."""
    
    if model_name=="gpt-3.5-turbo":
        if memory_max_token is None:
            memory_max_token = 1024 # max_tokens for 'gpt-3.5-turbo' = 4096
        memory = ConversationSummaryBufferMemory(
            max_token_limit=memory_max_token,
            llm=ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=openai_api_key,temperature=0.1),
            return_messages=True,
            memory_key='chat_history',
            output_key="answer",
            input_key="question"
        )
    else:
        memory = ConversationBufferMemory(
            return_messages=True,
            memory_key='chat_history',
            output_key="answer",
            input_key="question",
        )  
    return memory

In [246]:
memory = create_memory(model_name='gemini-pro',memory_max_token=20)

# save context
memory.save_context(
    inputs={"question":"what does DTC stand for?"},
    outputs={"answer":"""Diffuse to Choose (DTC) is a novel diffusion inpainting approach designed for the Vit-All application, 
    which allows users to virtually place any e-commerce item in any setting, ensuring detailed, semantically coherent blending with realistic 
    lighting and shadows. It effectively incorporates fine-grained cues from the reference image into the main U-Net decoder 
    using a secondary U-Net encoder.
    DTC can handle a variety of e-commerce products and can generate images using in-the-wild images & references. 
    It is superior to existing zero-shot personalization methods, especially in preserving the fine-grained details of items."""}
)
memory.save_context(
    inputs={"question":"what does Vit-all stand for?"},
    outputs={"answer":"Virtual Try-All"}
)

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='what does DTC stand for?'),
  AIMessage(content='Diffuse to Choose (DTC) is a novel diffusion inpainting approach designed for the Vit-All application, \n    which allows users to virtually place any e-commerce item in any setting, ensuring detailed, semantically coherent blending with realistic \n    lighting and shadows. It effectively incorporates fine-grained cues from the reference image into the main U-Net decoder \n    using a secondary U-Net encoder.\n    DTC can handle a variety of e-commerce products and can generate images using in-the-wild images & references. \n    It is superior to existing zero-shot personalization methods, especially in preserving the fine-grained details of items.'),
  HumanMessage(content='what does Vit-all stand for?'),
  AIMessage(content='Virtual Try-All')]}

In [247]:
standalone_question_template = """Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question, in its original language.\n\n
Chat History:\n{chat_history}\n
Follow Up Input: {question}\n
Standalone question:"""

standalone_question_prompt = PromptTemplate(
    input_variables=['chat_history', 'question'], 
    template=standalone_question_template
)

In [248]:

memory = create_memory(model_name='gemini-pro')
memory.save_context(
    inputs={"question":"what does DTC stand for?"}, 
    outputs={"answer":"DTC stands for Diffuse to choose."}
)

standalone_question_prompt.invoke(
    {"question":"plaese give more details about it, including its use cases and implementation.",
    "chat_history":memory.chat_memory})

StringPromptValue(text='Given the following conversation and a follow up question, \nrephrase the follow up question to be a standalone question, in its original language.\n\n\nChat History:\nHuman: what does DTC stand for?\nAI: DTC stands for Diffuse to choose.\n\nFollow Up Input: plaese give more details about it, including its use cases and implementation.\n\nStandalone question:')

In [249]:

def answer_template(language="english"):
    """Pass the standalone question along with the chat history and context 
    to the `LLM` wihch will answer"""
    
    template = f"""Answer the question at the end, using only the following context (delimited by <context></context>).
Your answer must be in the language at the end. 

<context>
{{chat_history}}

{{context}} 
</context>

Question: {{question}}

Language: {language}.
"""
    return template

In [250]:

answer_prompt = ChatPromptTemplate.from_template(answer_template())

# invoke the ChatPromptTemplate
answer_prompt.invoke(
    {"question":"plaese give more details about DTC, including its use cases and implementation.",
     "context":[Document(page_content="DTC use cases include...")], # the context is a list of retrieved documents.
     "chat_history":memory.chat_memory}
)

ChatPromptValue(messages=[HumanMessage(content="Answer the question at the end, using only the following context (delimited by <context></context>).\nYour answer must be in the language at the end. \n\n<context>\nHuman: what does DTC stand for?\nAI: DTC stands for Diffuse to choose.\n\n[Document(page_content='DTC use cases include...')] \n</context>\n\nQuestion: plaese give more details about DTC, including its use cases and implementation.\n\nLanguage: english.\n")])

In [251]:
# Here is an example using gemini-pro
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API token
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if the token is loaded correctly

# Create a Conversational Retrieval Chain using Mistral-7B-Instruct-v0.2
chain = ConversationalRetrievalChain.from_llm(
    condense_question_prompt=standalone_question_prompt,
    combine_docs_chain_kwargs={'prompt': answer_prompt},
    condense_question_llm=instantiate_LLM(
        LLM_provider="Google",api_key=google_api_key,temperature=0.1,
        model_name="gemini-pro"
    ),

    memory=create_memory("gemini-pro"),
    retriever = base_retriever_google, 
    llm=instantiate_LLM(
        LLM_provider="Google",api_key=google_api_key,temperature=0.5,
        model_name="gemini-pro"),
    chain_type= "stuff",
    verbose= False,
    return_source_documents=True   
)

# Example note: If using Gemini-Pro or other models, modify model_name accordingly.


In [255]:
# let's invoke the chain
response = chain.invoke({"question":"what is DTC?"})
response

{'question': 'what is DTC?',
 'chat_history': [HumanMessage(content='what does DTC stand for?'),
  AIMessage(content='The provided context does not mention what DTC stands for, so I cannot answer this question from the provided context.'),
  HumanMessage(content='plaese give more details about it, including its use cases and implementation.'),
  AIMessage(content='The provided context does not mention what DTC stands for or its use cases and implementation, so I cannot answer this question from the provided context.'),
  HumanMessage(content='what is DTC?'),
  AIMessage(content='The provided context does not mention what DTC stands for, so I cannot answer this question from the provided context.')],
 'answer': 'The provided context does not mention what DTC stands for, so I cannot answer this question from the provided context.',
 'source_documents': [Document(page_content='Figure 5. DTC can handle variety of e-commerce products and can generate images using in-the-wild images & refere

In [256]:
# Get chat history of the chain's memory

chain.memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='what does DTC stand for?'),
  AIMessage(content='The provided context does not mention what DTC stands for, so I cannot answer this question from the provided context.'),
  HumanMessage(content='plaese give more details about it, including its use cases and implementation.'),
  AIMessage(content='The provided context does not mention what DTC stands for or its use cases and implementation, so I cannot answer this question from the provided context.'),
  HumanMessage(content='what is DTC?'),
  AIMessage(content='The provided context does not mention what DTC stands for, so I cannot answer this question from the provided context.')]}

In [254]:

follow_up_question = "plaese give more details about it, including its use cases and implementation."

Markdown(chain.invoke({"question":follow_up_question})['answer'])

The provided context does not mention what DTC stands for or its use cases and implementation, so I cannot answer this question from the provided context.

In [257]:
from langchain.chains import ConversationalRetrievalChain

def create_ConversationalRetrievalChain(
    llm,condense_question_llm,
    retriever,
    chain_type= 'stuff',
    language="english",
    model_name='gpt-3.5-turbo'
):
    """Create a ConversationalRetrievalChain.
    First, it passes the follow-up question along with the chat history to an LLM which rephrases 
    the question and generates a standalone query. 
    This query is then sent to the retriever, which fetches relevant documents (context) 
    and passes them along with the standalone question and chat history to an LLM to answer.
    """
    
    # 1. Define the standalone_question prompt. 
    # Pass the follow-up question along with the chat history to the `condense_question_llm`
    # which rephrases the question and generates a standalone question.

    standalone_question_prompt = PromptTemplate(
        input_variables=['chat_history', 'question'], 
        template="""Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question, in its original language.\n\n
Chat History:\n{chat_history}\n
Follow Up Input: {question}\n
Standalone question:""")

    # 2. Define the answer_prompt
    # Pass the standalone question + the chat history + the context (retrieved documents) to the `LLM` wihch will answer
    
    answer_prompt = ChatPromptTemplate.from_template(answer_template(language=language))

    # 3. Add ConversationSummaryBufferMemory for gpt-3.5, and ConversationBufferMemory for the other models
    
    memory = create_memory(model_name)

    # 4. Create the ConversationalRetrievalChain

    chain = ConversationalRetrievalChain.from_llm(
        condense_question_prompt=standalone_question_prompt,
        combine_docs_chain_kwargs={'prompt': answer_prompt},
        condense_question_llm=condense_question_llm,

        memory=memory,
        retriever = retriever,
        llm=llm,

        chain_type= chain_type,
        verbose= False,
        return_source_documents=True    
    )

    print("Conversational retriever chain created successfully!")
    
    return chain,memory

In [258]:
from langchain.schema import format_document,Document
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [259]:
# We will use a simple `ConversationBufferMemory' here for simplicity.

memory = ConversationBufferMemory(memory_key="chat_history",output_key="answer", input_key="question",return_messages=True)
memory.load_memory_variables({})

{'chat_history': []}

In [260]:
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
)
print(loaded_memory.invoke({}))
print(loaded_memory.invoke({"x":1}))

{'chat_history': []}
{'x': 1, 'chat_history': []}


In [261]:
# 2. Pass the follow-up question along with the chat history to the LLM, and parse the answer (standalone_question).

condense_question_prompt = PromptTemplate(
    input_variables=['chat_history', 'question'], 
    template="""Given the following conversation and a follow up question (at the end), 
rephrase the follow up question to be a standalone question, in the same language as the follow up question.\n\n
Chat History:\n{chat_history}\n
Follow up question: {question}\n
Standalone question:"""
)

condense_question_llm = instantiate_LLM(
    LLM_provider="Google",api_key=google_api_key,temperature=0.1,
    model_name="gemini-pro"
)

standalone_question_chain = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | condense_question_prompt
    | condense_question_llm
    | StrOutputParser(),
}

# 3. Combine load_memory and standalone_question_chain
chain_question = loaded_memory | standalone_question_chain

In [262]:

# Here is an example of how to invoke chain_question:

memory.clear()
memory.save_context(
    {"question": "What does DTC stand for?"},
    {"answer": "Diffuse to Choose."}
)

print("Chat history:\n",memory.load_memory_variables({}))
print("\nFollow-up question:\n","plaese give more details about it, including its use cases and implementation.")

# invoke chain_question
response = chain_question.invoke({"question":"plaese give more details about it, including its use cases and implementation."})['standalone_question']
print("\nStandalone_question:\n",response)

Chat history:
 {'chat_history': [HumanMessage(content='What does DTC stand for?'), AIMessage(content='Diffuse to Choose.')]}

Follow-up question:
 plaese give more details about it, including its use cases and implementation.

Standalone_question:
 Can you provide more information about Diffuse to Choose (DTC), including its use cases and implementation?


In [263]:
# Create retriever 
retriever_Google = retrieval_blocks(
    create_vectorstore=False,
    LLM_service="Google",
    vectorstore_name="Vit_All_Google_Embeddings",
    retriever_type="Cohere_reranker",
    base_retriever_search_type="similarity", base_retriever_k=12,
    compression_retriever_k=16,
    cohere_api_key=cohere_api_key,cohere_top_n=10,
)


Cohere_reranker is created successfully!
Relevant documents will be retrieved from vectorstore (Vit_All_Google_Embeddings) which uses Google embeddings and has 585 chunks.


In [264]:
from langchain.schema import Document

def _combine_documents(docs, document_prompt, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Retrieve relevant documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever_Google,
    "question": lambda x: x["standalone_question"],
}

# Get variables ['chat_history', 'context', 'question'] that will be passed to `answer_prompt`

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
answer_prompt = ChatPromptTemplate.from_template(answer_template()) # 3 variables are expected ['chat_history', 'context', 'question']

answer_prompt_variables = {
    "context": lambda x: _combine_documents(docs=x["docs"],document_prompt=DEFAULT_DOCUMENT_PROMPT),
    "question": itemgetter("question"),
    "chat_history": itemgetter("chat_history") # get chat_history from `loaded_memory` variable
}

# Load memory, format `answer_prompt` with variables (context, question and chat_history) and pass the `answer_prompt to LLM.
# return answer, docs and standalone_question

llm = instantiate_LLM(
    LLM_provider="Google",api_key=google_api_key,temperature=0.5,
    model_name="gemini-pro"
)

chain_answer = {
    "answer": loaded_memory | answer_prompt_variables | answer_prompt | llm,
    "docs": lambda x: [
        Document(page_content=doc.page_content,metadata=doc.metadata) # return only page_content and metadata (_DocumentWithState returns them + state.)
        for doc in x["docs"]
    ], 
    "standalone_question": lambda x:x["question"] # return standalone_question
}

In [265]:

conversational_retriever_chain = chain_question | retrieved_documents | chain_answer

In [266]:

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What does DTC stand for?'),
  AIMessage(content='Diffuse to Choose.')]}

In [267]:

follow_up_question = "plaese give more details about it, including its use cases and implementation."

response = conversational_retriever_chain.invoke({"question":follow_up_question})
Markdown(response['answer'].content)

The provided context does not contain any information about the use cases or implementation of Diffuse to Choose (DTC).

In [268]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What does DTC stand for?'),
  AIMessage(content='Diffuse to Choose.')]}

In [269]:

memory.save_context( {"question": follow_up_question}, {"answer": response['answer'].content} ) # update memory

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What does DTC stand for?'),
  AIMessage(content='Diffuse to Choose.'),
  HumanMessage(content='plaese give more details about it, including its use cases and implementation.'),
  AIMessage(content='The provided context does not contain any information about the use cases or implementation of Diffuse to Choose (DTC).')]}

In [270]:
def answer_template(language="english"):
    """Pass the standalone question along with the chat history and context (retrieved documents) to the `LLM` to get an answer."""
    
    template = f"""Answer the question at the end, using only the following context (delimited by <context></context>).
Your answer must be in the language at the end. 

<context>
{{chat_history}}

{{context}} 
</context>

Question: {{question}}

Language: {language}.
"""
    return template
    
def _combine_documents(docs, document_prompt, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

def custom_ConversationalRetrievalChain(
    llm,condense_question_llm,
    retriever,
    language="english",
    llm_provider="OpenAI",
    model_name='gpt-3.5-turbo',
):
    """Create a ConversationalRetrievalChain step by step.
    """
    ##############################################################
    # Step 1: Create a standalone_question chain
    ##############################################################
    
    # 1. Create memory: ConversationSummaryBufferMemory for gpt-3.5, and ConversationBufferMemory for the other models
    
    memory = create_memory(model_name)
    # memory = ConversationBufferMemory(memory_key="chat_history",output_key="answer", input_key="question",return_messages=True)

    # 2. load memory using RunnableLambda. Retrieves the chat_history attribute using itemgetter.
    loaded_memory = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
    )

    # 3. Pass the follow-up question along with the chat history to the LLM, and parse the answer (standalone_question).

    condense_question_prompt = PromptTemplate(
        input_variables=['chat_history', 'question'], 
        template = """Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question, in the same language as the follow up question.\n\n
Chat History:\n{chat_history}\n
Follow Up Input: {question}\n
Standalone question:"""        
)
        
    standalone_question_chain = {
        "standalone_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: get_buffer_string(x["chat_history"]),
        }
        | condense_question_prompt
        | condense_question_llm
        | StrOutputParser(),
    }
    
    # 4. Combine load_memory and standalone_question_chain
    chain_question = loaded_memory | standalone_question_chain
    
    ####################################################################################
    #   Step 2: Retrieve documents, pass them to the LLM, and return the response.
    ####################################################################################

    # 5. Retrieve relevant documents
    retrieved_documents = {
        "docs": itemgetter("standalone_question") | retriever,
        "question": lambda x: x["standalone_question"],
    }
    
    # 6. Get variables ['chat_history', 'context', 'question'] that will be passed to `answer_prompt`
    
    DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
    answer_prompt = ChatPromptTemplate.from_template(answer_template(language=language)) 
    # 3 variables are expected ['chat_history', 'context', 'question'] by the ChatPromptTemplate   
    answer_prompt_variables = {
        "context": lambda x: _combine_documents(docs=x["docs"],document_prompt=DEFAULT_DOCUMENT_PROMPT),
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history") # get it from `loaded_memory` variable
    }
    
    # 7. Load memory, format `answer_prompt` with variables (context, question and chat_history) and pass the `answer_prompt to LLM.
    # return answer, docs and standalone_question
    
    chain_answer = {
        "answer": loaded_memory | answer_prompt_variables | answer_prompt | llm,
        # return only page_content and metadata 
        "docs": lambda x: [Document(page_content=doc.page_content,metadata=doc.metadata) for doc in x["docs"]],
        "standalone_question": lambda x:x["question"] # return standalone_question
    }

    # 8. Final chain
    conversational_retriever_chain = chain_question | retrieved_documents | chain_answer

    print("Conversational retriever chain created successfully!")

    return conversational_retriever_chain,memory

In [271]:
# # 

# QUESTIONS
questions = ["what does DTC stands for?",
             "plaese give more details about it, including its use cases and implementation.",
             "does it outperform other diffusion-based models? explain in details.",
             "what is Langchain?"]

In [272]:

# Instantiate the retriever and the ConversationalRetrievalChain :

retriever_Google = retrieval_blocks(
    create_vectorstore=False,
    LLM_service="Google",
    vectorstore_name="Vit_All_Google_Embeddings",
    retriever_type="Cohere_reranker",
    base_retriever_search_type="similarity", base_retriever_k=12,
    compression_retriever_k=16,
    cohere_api_key=cohere_api_key,cohere_top_n=10,
)

chain_gemini,memory_gemini = custom_ConversationalRetrievalChain(
    llm = instantiate_LLM(
        LLM_provider="Google",api_key=google_api_key,temperature=0.5,model_name="gemini-pro"
    ),
    condense_question_llm = instantiate_LLM(
        LLM_provider="Google",api_key=google_api_key,temperature=0.1,model_name="gemini-pro"),
    retriever=retriever_Google,
    language="english",
    llm_provider="Google",
    model_name="gemini-pro"
)


Cohere_reranker is created successfully!
Relevant documents will be retrieved from vectorstore (Vit_All_Google_Embeddings) which uses Google embeddings and has 585 chunks.
Conversational retriever chain created successfully!


In [273]:
%%time

memory_gemini.clear()

for i,question in enumerate(questions):
    response = chain_gemini.invoke({"question":question})
    answer = response['answer'].content
    print(f"Question[{i}]:",question)
    print("Standalone_question:",response['standalone_question'])
    print("Answer:\n",answer,f"\n\n{'-' * 100}\n")
    
    memory_gemini.save_context( {"question": question}, {"answer": answer} ) # update memory

Question[0]: what does DTC stands for?
Standalone_question: What is the meaning of DTC?
Answer:
 The provided context does not mention the meaning of DTC, so I cannot answer this question from the provided context. 

----------------------------------------------------------------------------------------------------

Question[1]: plaese give more details about it, including its use cases and implementation.
Standalone_question: What are the details of DTC, including its use cases and implementation?
Answer:
 The provided context does not mention the details of DTC, including its use cases and implementation, so I cannot answer this question from the provided context. 

----------------------------------------------------------------------------------------------------

Question[2]: does it outperform other diffusion-based models? explain in details.
Standalone_question: How does DTC compare to other diffusion-based models in terms of performance?
Answer:
 The provided context does not 